In [3]:
!pip install mlxtend

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import os

In [6]:
os.chdir('C:\\Users\\shoun\\LetsUpgrade\\Datasets')

In [11]:
df = pd.read_csv('OnlineRetail.csv', encoding='unicode_escape')

In [15]:
df.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [17]:
'''Check Information of your dataset'''
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [68]:
'''Convert Date-Time format to separate year & month format'''

# Converting into proper date time format
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

df['year'] = df['InvoiceDate'].dt.year
df['month'] = df['InvoiceDate'].dt.month
df['hour'] = df['InvoiceDate'].dt.hour

In [69]:
'''Check for missing values using isnull().sum() '''
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
year                0
month               0
hour                0
dtype: int64

In [70]:
'''Check number of unique values for every variable (column)'''
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
year               2
month             12
hour              15
dtype: int64

In [71]:
'''Check for negative values of Quantity and remove the negative quantity'''
df1 = df[df['Quantity'] > 0]

In [72]:
df1

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,hour
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,2010,12,8
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,8
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,2010,12,8
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,8
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,2010,12,8
...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,2011,12,12
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011,12,12
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,12
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011,12,12


In [73]:
'''Checking if there are any invoice with 'C' in invoice number '''
df1['InvoiceNo'] = df1['InvoiceNo'].astype('str')
df1['InvoiceNo'].str.contains('C').value_counts()

<ipython-input-73-b5ee5eda50d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['InvoiceNo'] = df1['InvoiceNo'].astype('str')


False    531285
Name: InvoiceNo, dtype: int64

In [74]:
'''Check if there are any null values'''
df1.isnull().any()

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
year           False
month          False
hour           False
dtype: bool

In [75]:
'''Fill null values in CustomerID and Description column'''
df1['CustomerID'] = df1['CustomerID'].fillna('00000')
df1['Description'] = df1['Description'].fillna('unknown')

<ipython-input-75-a5e6c9c10f91>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['CustomerID'] = df1['CustomerID'].fillna('00000')
<ipython-input-75-a5e6c9c10f91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Description'] = df1['Description'].fillna('unknown')


In [77]:
'''Check for revenue earnings from every transaction'''
df1['Revenue'] = round(df1['Quantity']*df1['UnitPrice'], 2)

<ipython-input-77-e3528f5db9b3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Revenue'] = round(df1['Quantity']*df1['UnitPrice'], 2)


In [82]:
df1.nunique()

InvoiceNo      20728
StockCode       3941
Description     4078
Quantity         393
InvoiceDate    19052
UnitPrice       1293
CustomerID      4340
Country           38
year               2
month             12
hour              15
Revenue         4598
dtype: int64

In [87]:
df1.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,hour,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,2010,12,8,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,2010,12,8,20.34


In [84]:
'''Group products with invoice No.'''
products_group = df1[['InvoiceNo', 'StockCode']].groupby('InvoiceNo').count()

In [104]:
products_group.head(2)

,StockCode
InvoiceNo,
536365,7
536366,2


In [105]:
'''Find unique countries'''
df1['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Bahrain', 'Israel', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [107]:
df1['Country'].value_counts()

United Kingdom          486286
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

In [109]:
'''group invoice No. countrywise'''
country_group = df1[['InvoiceNo', 'Country']].groupby('Country').count()

In [213]:
country_group.sort_values(by = 'InvoiceNo',ascending=False)

,InvoiceNo
Country,
United Kingdom,486286
Germany,9042
France,8408
EIRE,7894
Spain,2485
Netherlands,2363
Belgium,2031
Switzerland,1967
Portugal,1501


In [116]:
'''Check counts of products'''
df1['Description'].unique()

array(['WHITE HANGING HEART T-LIGHT HOLDER', 'WHITE METAL LANTERN',
       'CREAM CUPID HEARTS COAT HANGER', ..., 'LETTER "U" BLING KEY RING',
       'CREAM HANGING HEART T-LIGHT HOLDER',
       'PAPER CRAFT , LITTLE BIRDIE'], dtype=object)

In [114]:
df1['Description'].value_counts()

WHITE HANGING HEART T-LIGHT HOLDER    2327
JUMBO BAG RED RETROSPOT               2115
REGENCY CAKESTAND 3 TIER              2019
PARTY BUNTING                         1707
LUNCH BAG RED RETROSPOT               1594
                                      ... 
PINK BOUDOIR T-LIGHT HOLDER              1
WRAP BLUE RUSSIAN FOLKART                1
LARGE ROUND CUTGLASS CANDLESTICK         1
HEAVENS SCENT FRAGRANCE OILS ASSTD       1
FRYING PAN RED POLKADOT                  1
Name: Description, Length: 4078, dtype: int64

# Using Apriori Algorithm

In [215]:
df2 = df1[df1['Country'] == 'Germany']

In [216]:
'''Since we need only Invoice number, Decription and Quantity, we can drop the rest of the rows'''
data = df2[['InvoiceNo','Description','Quantity']]

In [217]:
data = data.groupby(['InvoiceNo','Description']).sum()

In [218]:
data

Quantity
InvoiceNo Description                                  
536527    3 HOOK HANGER MAGIC GARDEN                 12
          5 HOOK HANGER MAGIC TOADSTOOL              12
          5 HOOK HANGER RED MAGIC TOADSTOOL          12
          ASSORTED COLOUR LIZARD SUCTION HOOK        24
          CHILDREN'S CIRCUS PARADE MUG               12
...                                                 ...
581578    SPOTTY BUNTING                              9
          VINTAGE DONKEY TAIL GAME                    6
          WRAP ALPHABET POSTER                       25
          WRAP CIRCUS PARADE                         25
          WRAP RED APPLES                            25

[9015 rows x 1 columns]

In [219]:
data2 = df1[['InvoiceNo','Description','Quantity']]
data2 = data2.groupby(['InvoiceNo','Description']).count()

In [220]:
data2

Quantity
InvoiceNo Description                                  
536365    CREAM CUPID HEARTS COAT HANGER              1
          GLASS STAR FROSTED T-LIGHT HOLDER           1
          KNITTED UNION FLAG HOT WATER BOTTLE         1
          RED WOOLLY HOTTIE WHITE HEART.              1
          SET 7 BABUSHKA NESTING BOXES                1
...                                                 ...
581587    PLASTERS IN TIN STRONGMAN                   1
          SPACEBOY LUNCH BOX                          1
A563185   Adjust bad debt                             1
A563186   Adjust bad debt                             1
A563187   Adjust bad debt                             1

[520566 rows x 1 columns]

In [221]:
ref_data = data.unstack().fillna(0)

In [222]:
ref_data.isnull().any()

          Description                      
Quantity   50'S CHRISTMAS GIFT BAG LARGE       False
           DOLLY GIRL BEAKER                   False
           I LOVE LONDON MINI BACKPACK         False
           RED SPOT GIFT BAG LARGE             False
           SET 2 TEA TOWELS I LOVE LONDON      False
                                               ...  
          ZINC HEART LATTICE T-LIGHT HOLDER    False
          ZINC METAL HEART DECORATION          False
          ZINC T-LIGHT HOLDER STAR LARGE       False
          ZINC T-LIGHT HOLDER STARS SMALL      False
          ZINC WILLIE WINKIE  CANDLE STICK     False
Length: 1697, dtype: bool

In [223]:
def valtozero(i):
    if i > 0:
        return 1
    else:
        return 0

In [224]:
ult_data = ref_data.applymap(valtozero)

In [225]:
ult_data.isnull().any()

          Description                      
Quantity   50'S CHRISTMAS GIFT BAG LARGE       False
           DOLLY GIRL BEAKER                   False
           I LOVE LONDON MINI BACKPACK         False
           RED SPOT GIFT BAG LARGE             False
           SET 2 TEA TOWELS I LOVE LONDON      False
                                               ...  
          ZINC HEART LATTICE T-LIGHT HOLDER    False
          ZINC METAL HEART DECORATION          False
          ZINC T-LIGHT HOLDER STAR LARGE       False
          ZINC T-LIGHT HOLDER STARS SMALL      False
          ZINC WILLIE WINKIE  CANDLE STICK     False
Length: 1697, dtype: bool

### While applying Apriori, set minumum support beforeheand. In this example, I will consider it as 0.05

In [233]:
support = apriori(ult_data, min_support = 0.1, use_colnames=True)

In [234]:
support

,support,itemsets
0,0.102845,"((Quantity, 6 RIBBONS RUSTIC CHARM))"
1,0.100656,"((Quantity, JUMBO BAG WOODLAND ANIMALS))"
2,0.115974,"((Quantity, PLASTERS IN TIN CIRCUS PARADE ))"
3,0.107221,"((Quantity, PLASTERS IN TIN SPACEBOY))"
4,0.137856,"((Quantity, PLASTERS IN TIN WOODLAND ANIMALS))"
5,0.818381,"((Quantity, POSTAGE))"
6,0.137856,"((Quantity, REGENCY CAKESTAND 3 TIER))"
7,0.157549,"((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ))"
8,0.245077,"((Quantity, ROUND SNACK BOXES SET OF4 WOODLAND ))"
9,0.102845,"((Quantity, SPACEBOY LUNCH BOX ))"


### Once Support is done, we will check for Association Rules which will give us confidence and lift

In [237]:
association = association_rules(support, metric='lift', min_threshold=2)

In [238]:
association

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ))","((Quantity, ROUND SNACK BOXES SET OF4 WOODLAND ))",0.157549,0.245077,0.131291,0.833333,3.400298,0.092679,4.529540
1,"((Quantity, ROUND SNACK BOXES SET OF4 WOODLAND ))","((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ))",0.245077,0.157549,0.131291,0.535714,3.400298,0.092679,1.814509
2,"((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ...","((Quantity, ROUND SNACK BOXES SET OF4 WOODLAND ))",0.150985,0.245077,0.124726,0.826087,3.370730,0.087724,4.340810
3,"((Quantity, POSTAGE), (Quantity, ROUND SNACK B...","((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ))",0.225383,0.157549,0.124726,0.553398,3.512540,0.089218,1.886357
4,"((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ))","((Quantity, POSTAGE), (Quantity, ROUND SNACK B...",0.157549,0.225383,0.124726,0.791667,3.512540,0.089218,3.718162
5,"((Quantity, ROUND SNACK BOXES SET OF4 WOODLAND ))","((Quantity, ROUND SNACK BOXES SET OF 4 FRUITS ...",0.245077,0.150985,0.124726,0.508929,3.370730,0.087724,1.728904


In [230]:
df2

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,year,month,hour,Revenue
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,2010-12-01 13:04:00,2.95,12662,Germany,2010,12,13,17.70
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,2010-12-01 13:04:00,2.55,12662,Germany,2010,12,13,15.30
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,2010-12-01 13:04:00,0.85,12662,Germany,2010,12,13,10.20
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 13:04:00,1.65,12662,Germany,2010,12,13,19.80
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,2010-12-01 13:04:00,1.95,12662,Germany,2010,12,13,23.40
...,...,...,...,...,...,...,...,...,...,...,...,...
541801,581578,22993,SET OF 4 PANTRY JELLY MOULDS,12,2011-12-09 12:16:00,1.25,12713,Germany,2011,12,12,15.00
541802,581578,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,2011-12-09 12:16:00,0.85,12713,Germany,2011,12,12,10.20
541803,581578,22908,PACK OF 20 NAPKINS RED APPLES,12,2011-12-09 12:16:00,0.85,12713,Germany,2011,12,12,10.20
541804,581578,23215,JINGLE BELL HEART ANTIQUE SILVER,12,2011-12-09 12:16:00,2.08,12713,Germany,2011,12,12,24.96
